In [1]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, AutoConfig, GPTNeoForCausalLM
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import os
import openai
from re import search

/work/anaseh_umass_edu/anaconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.device_count()

1

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")

In [4]:
df = pd.read_json('arxiv-metadata-oai-snapshot.json', lines=True)

In [ ]:
# To check the length of the texts

max = 0
num = 0
for i in range(len(df)):
    length = len(tokenizer(df["abstract"][i]).input_ids)
    if length > max:
        max = length
    if length > 1024:
        num += 1

10000
50000
200000
500000


In [6]:
df.head(5)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [5]:
# To get the list of all categories

cats = []
for i in range(len(df)):
    if df['categories'][i] not in cats:
        cats.append(df['categories'][i])
        
new_cats = []
all_cats = list(set(cats))
for i in range(len(all_cats)):
    splitted = all_cats[i].split(" ")
    for j in range(len(splitted)):
        if splitted[j] not in new_cats:
            new_cats.append(splitted[j])

10000
50000
200000
500000
1000000
1500000


In [7]:
print(len(new_cats))
print(new_cats)

176
['physics.comp-ph', 'math.NA', 'stat.CO', 'eess.SY', 'cs.AI', 'cs.LG', 'cs.SY', 'stat.ME', 'nlin.CD', 'nlin.AO', 'physics.bio-ph', 'astro-ph.SR', 'physics.data-an', 'physics.space-ph', 'eess.SP', 'cs.IT', 'math.IT', 'cs.HC', 'cs.NI', 'cs.SI', 'q-bio.BM', 'cs.GR', 'hep-th', 'astro-ph.CO', 'cond-mat.supr-con', 'hep-ph', 'math-ph', 'math.MP', 'cs.MA', 'cs.RO', 'quant-ph', 'nlin.SI', 'solv-int', 'cond-mat.str-el', 'cond-mat.mes-hall', 'cond-mat.mtrl-sci', 'cs.CV', 'cs.CG', 'cs.SC', 'physics.chem-ph', 'physics.plasm-ph', 'math.CO', 'cs.DM', 'cs.DS', 'math.PR', 'stat.ML', 'hep-lat', 'cs.CL', 'cs.FL', 'cs.CR', 'cs.CY', 'cs.NE', 'cs.DC', 'physics.flu-dyn', 'math.OC', 'math.CA', 'math.AP', 'math.CV', 'math.FA', 'eess.IV', 'physics.med-ph', 'q-bio.GN', 'math.AC', 'astro-ph', 'physics.class-ph', 'physics.geo-ph', 'astro-ph.IM', 'stat.AP', 'math.DS', 'q-bio.PE', 'q-bio.QM', 'cs.GT', 'math.DG', 'cs.ET', 'cs.CC', 'nlin.CG', 'cs.NA', 'math.GT', 'q-bio.MN', 'math.NT', 'q-fin.ST', 'cond-mat.stat-me

In [8]:
# To count the number of papers of each category

new_cats_dict = {}
for i in range(len(new_cats)):
    new_cats_dict[new_cats[i]] = 0

for i in range(len(df)):
    splitted = df['categories'][i].split()
    new_cats_dict[splitted[0]] += 1


{'physics.comp-ph': 0, 'math.NA': 0, 'stat.CO': 0, 'eess.SY': 0, 'cs.AI': 0, 'cs.LG': 0, 'cs.SY': 0, 'stat.ME': 0, 'nlin.CD': 0, 'nlin.AO': 0, 'physics.bio-ph': 0, 'astro-ph.SR': 0, 'physics.data-an': 0, 'physics.space-ph': 0, 'eess.SP': 0, 'cs.IT': 0, 'math.IT': 0, 'cs.HC': 0, 'cs.NI': 0, 'cs.SI': 0, 'q-bio.BM': 0, 'cs.GR': 0, 'hep-th': 0, 'astro-ph.CO': 0, 'cond-mat.supr-con': 0, 'hep-ph': 0, 'math-ph': 0, 'math.MP': 0, 'cs.MA': 0, 'cs.RO': 0, 'quant-ph': 0, 'nlin.SI': 0, 'solv-int': 0, 'cond-mat.str-el': 0, 'cond-mat.mes-hall': 0, 'cond-mat.mtrl-sci': 0, 'cs.CV': 0, 'cs.CG': 0, 'cs.SC': 0, 'physics.chem-ph': 0, 'physics.plasm-ph': 0, 'math.CO': 0, 'cs.DM': 0, 'cs.DS': 0, 'math.PR': 0, 'stat.ML': 0, 'hep-lat': 0, 'cs.CL': 0, 'cs.FL': 0, 'cs.CR': 0, 'cs.CY': 0, 'cs.NE': 0, 'cs.DC': 0, 'physics.flu-dyn': 0, 'math.OC': 0, 'math.CA': 0, 'math.AP': 0, 'math.CV': 0, 'math.FA': 0, 'eess.IV': 0, 'physics.med-ph': 0, 'q-bio.GN': 0, 'math.AC': 0, 'astro-ph': 0, 'physics.class-ph': 0, 'physics.

In [10]:
print(new_cats_dict)

{'math.NA': 32024, 'cs.LG': 133001, 'cs.NA': 18040, 'cs.DS': 21137, 'physics.data-an': 9434, 'physics.comp-ph': 19920, 'physics.geo-ph': 6202, 'cs.OS': 753, 'cs.DC': 17961, 'cs.NI': 19296, 'cs.PF': 3346, 'physics.chem-ph': 19234, 'astro-ph.EP': 25527, 'astro-ph.IM': 25313, 'physics.optics': 40663, 'cs.SE': 12685, 'eess.SP': 21009, 'physics.bio-ph': 13800, 'cs.NE': 12166, 'nlin.AO': 6577, 'q-bio': 1356, 'cs.CV': 90534, 'nlin.CG': 1420, 'math.PR': 50630, 'physics.soc-ph': 20040, 'stat.ML': 58257, 'q-fin.PM': 1593, 'cond-mat.dis-nn': 21884, 'math.OC': 40749, 'q-fin.RM': 1991, 'math.RT': 23729, 'math.AP': 54861, 'math.MG': 10086, 'math.SP': 8849, 'hep-ph': 170301, 'astro-ph.GA': 56457, 'astro-ph.HE': 50768, 'astro-ph.SR': 54554, 'gr-qc': 98696, 'hep-th': 156801, 'math.GT': 20918, 'math.QA': 17838, 'alg-geom': 1423, 'dg-ga': 732, 'math.AG': 48175, 'math.DG': 38608, 'cs.AI': 59217, 'cs.RO': 22971, 'cs.IR': 13257, 'cs.CE': 5560, 'physics.med-ph': 5795, 'math-ph': 73274, 'math.MP': 73274, 'phy

In [10]:
new_cats_df = pd.DataFrame.from_dict(new_cats_dict, orient='index')
new_cats_df.columns = ["Freq"]

In [11]:
new_cats_df.head(5)

,Freq
physics.comp-ph,6951
math.NA,23816
stat.CO,3251
eess.SY,9180
cs.AI,16677


In [12]:
new_cats_df = new_cats_df.sort_values(by='Freq', ascending=False)

In [13]:
new_cats_df.to_csv("main_categories.csv")

In [34]:
# To filter the selected categories and create the a large dataset of all papers of these categories

my_indices = []
my_cats = ['cs.CR', 'eess.SP', 'stat.ME', 'gr-qc', 'physics.optics']
for i in range(len(df)):
    splitted = df['categories'][i].split(" ")
    if splitted[0] in my_cats:
        my_indices.append(i)
        
selected_rows = df.loc[my_indices]
selected_rows = selected_rows.reset_index(drop=True)
selected_rows.to_csv('selected_papers_for_exp.csv')

10000
50000
200000
500000
1000000
1500000
2000000


In [ ]:
df = pd.read_csv('selected_papers_for_exp.csv')

In [ ]:
def filter_func(abstract):
    
    length = len(tokenizer(abstract).input_ids)
    
    if length > 170 and length < 1023:
        return True
    else:
        return False

In [7]:
def filter_func2(categories, target_category):
    if categories.split(" ")[0] == target_category:
        return True
    else:
        return False

In [ ]:
# Create datasets with different sizes

my_cats = ['cs.CR', 'eess.SP', 'stat.ME', 'gr-qc', 'physics.optics']
for i in range(len(my_cats)):
    if i == 0:
        filtered_df = df.loc[df['categories'].apply(lambda x: filter_func2(x, my_cats[i]))]
        filtered_df = filtered_df.drop('Unnamed: 0', axis=1)
        filtered_df = filtered_df.reset_index(drop=True)
        filtered_df = filtered_df.loc[filtered_df['abstract'].apply(lambda x: filter_func(x))]
        filtered_df = filtered_df.reset_index(drop=True)
        filtered_df = filtered_df.sample(n=1000)
        filtered_df = filtered_df.reset_index(drop=True)
        final_dataframe = filtered_df[['abstract']]
        final_dataframe = final_dataframe.applymap(lambda x: '<' + my_cats[i] + '>: ' + str(x))
    else:
        filtered_df = df.loc[df['categories'].apply(lambda x: filter_func2(x, my_cats[i]))]
        filtered_df = filtered_df.drop('Unnamed: 0', axis=1)
        filtered_df = filtered_df.reset_index(drop=True)
        filtered_df = filtered_df.loc[filtered_df['abstract'].apply(lambda x: filter_func(x))]
        filtered_df = filtered_df.reset_index(drop=True)
        filtered_df = filtered_df.sample(n=1000)
        filtered_df = filtered_df.reset_index(drop=True)
        temp_dataframe = filtered_df[['abstract']]
        temp_dataframe = temp_dataframe.applymap(lambda x: '<' + my_cats[i] + '>: ' + str(x))
        frames = [final_dataframe, temp_dataframe]
        final_dataframe = pd.concat(frames, ignore_index=True)

In [25]:
final_dataframe.head(5)

,abstract
0,<cs.CR>: The world has been experiencing a m...
1,<cs.CR>: We explore the additional security ...
2,<cs.CR>: Machine learning (ML) has become a ...
3,<cs.CR>: The smart grid incentivizes distrib...
4,<cs.CR>: A physical unclonable function (PUF...


In [28]:
print(len(final_dataframe))

5000


In [27]:
final_dataframe.columns = ['text']
final_dataframe.head(4)

,text
0,<cs.CR>: The world has been experiencing a m...
1,<cs.CR>: We explore the additional security ...
2,<cs.CR>: Machine learning (ML) has become a ...
3,<cs.CR>: The smart grid incentivizes distrib...


In [30]:
print(final_dataframe.shape)

(5000, 1)


In [31]:
final_dataframe.to_csv('equal_dataset_5000.csv')